# Notebook Title

## Setup Python and R environment
you can ignore this section

In [92]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [94]:
%%R

# My commonly used R imports

require('tidyverse')

## Load & Clean Data

👉 Load the data along with the census connectors below (the output of the `connect-to-census.ipynb` notebook) and do any cleanup you'd like to do.

In [95]:
import pandas as pd

In [96]:
df_merge2.columns.to_list()

['Unnamed: 0',
 'Borough',
 'Borough/Citywide Office (B/CO)',
 'District',
 'School',
 'School Name_x',
 'School Category',
 'Program',
 'Language',
 'Language (Translated)',
 'General/Special Education',
 'Special Education Model',
 'fiscal_year',
 'system_code',
 'location_code',
 'location_name',
 'BEDS',
 'Managed_by_name',
 'location_type_description',
 'Location_Category_Description',
 'Grades_text',
 'Grades_final_text',
 'open_date',
 'Status_descriptions',
 'Primary_building_code',
 'primary_address_line_1',
 'State_code',
 'X_COORDINATE',
 'Y_COORDINATE',
 'LONGITUDE',
 'LATITUDE',
 'Community_district',
 'Council-district',
 'Census_tract',
 'Borough_block_lot',
 'NTA',
 'NTA_Name',
 'Principal_Name',
 'Principal_title',
 'Principal_phone_number',
 'fax_number',
 'Geographical_District_code',
 'Administrative_District_Code',
 'Administrative_District_Name',
 'community_school_sup_name',
 'Tier_3_Support_Location_Name',
 'Tier_3_Support_Leader_Name',
 'Tier_2_Support_Location

In [97]:
df_merge2 = pd.read_csv('schoolData.csv', dtype={'TRACT': str, 'COUNTY': str, 'STATE': str})
df_merge2[['STATE', 'COUNTY', 'TRACT', 'GEOID']]

,STATE,COUNTY,TRACT,GEOID
0,36,061,003001,360610030013002
1,36,061,003001,360610030013002
2,36,061,003001,360610030013002
3,36,061,003001,360610030013002
4,36,061,003001,360610030013002
...,...,...,...,...
2746,36,005,012300,360050123003000
2747,36,005,012300,360050123003000
2748,36,005,012300,360050123003000
2749,36,005,012300,360050123003000


In [98]:
df_merge2['mergeid']= df_merge2['STATE'] + df_merge2['COUNTY'] + df_merge2['TRACT'] 
df_merge2['mergeid']

0       36061003001
1       36061003001
2       36061003001
3       36061003001
4       36061003001
           ...     
2746    36005012300
2747    36005012300
2748    36005012300
2749    36005012300
2750    36005012300
Name: mergeid, Length: 2751, dtype: object

## 👉 Grab Census Data

1. loading the Census API key

In [99]:
import dotenv

# Load the environment variables
# (loads CENSUS_API_KEY from .env)
dotenv.load_dotenv()


True

In [100]:
!touch .env

In [101]:
%%R 

require('tidycensus')

# because it an environment variable, we don't have to 
# explicitly pass this string to R, it is readable here
# in this R cell.
census_api_key(Sys.getenv("CENSUS_API_KEY"))

To install your API key for use in future sessions, run this function with `install = TRUE`.


2. Decide which Census variables you want

    Use <https://censusreporter.org/> to figure out which tables you want. (if censusreporter is down, check out the code in the cell below)

    -   Scroll to the bottom of the page to see the tables.
    -   If you already know the table ID, stick that in the "Explore" section to learn more about that table.

    By default this code loads (B01003_001) which we found in censusreporter here: https://censusreporter.org/tables/B01003/

    - find some other variables that you're also interested in
    - don't forget to pick a geography like "tract", "county" or "block group". here is the list of [all geographies](https://walker-data.com/tidycensus/articles/basic-usage.html#geography-in-tidycensus
    ).


In [102]:
%%R 
B03002_vars <- load_variables(2021, "acs5", cache = TRUE) %>%
    # Check if name contains B03002
    filter(str_detect(name, "B03002"))

# Print all rows of the filtered data
print(B03002_vars, n = Inf)

# A tibble: 21 × 4
   name       label                                            concept geography
   <chr>      <chr>                                            <chr>   <chr>    
 1 B03002_001 Estimate!!Total:                                 HISPAN… block gr…
 2 B03002_002 Estimate!!Total:!!Not Hispanic or Latino:        HISPAN… block gr…
 3 B03002_003 Estimate!!Total:!!Not Hispanic or Latino:!!Whit… HISPAN… block gr…
 4 B03002_004 Estimate!!Total:!!Not Hispanic or Latino:!!Blac… HISPAN… block gr…
 5 B03002_005 Estimate!!Total:!!Not Hispanic or Latino:!!Amer… HISPAN… block gr…
 6 B03002_006 Estimate!!Total:!!Not Hispanic or Latino:!!Asia… HISPAN… block gr…
 7 B03002_007 Estimate!!Total:!!Not Hispanic or Latino:!!Nati… HISPAN… block gr…
 8 B03002_008 Estimate!!Total:!!Not Hispanic or Latino:!!Some… HISPAN… block gr…
 9 B03002_009 Estimate!!Total:!!Not Hispanic or Latino:!!Two … HISPAN… block gr…
10 B03002_010 Estimate!!Total:!!Not Hispanic or Latino:!!Two … HISPAN… block gr…
11 B03002

In [103]:
%%R 

B03002_vars <- load_variables(2021, "acs5", cache = TRUE)

# Print all columns of B03002_vars
B03002_vars


# A tibble: 27,886 × 4
   name        label                                    concept        geography
   <chr>       <chr>                                    <chr>          <chr>    
 1 B01001A_001 Estimate!!Total:                         SEX BY AGE (W… tract    
 2 B01001A_002 Estimate!!Total:!!Male:                  SEX BY AGE (W… tract    
 3 B01001A_003 Estimate!!Total:!!Male:!!Under 5 years   SEX BY AGE (W… tract    
 4 B01001A_004 Estimate!!Total:!!Male:!!5 to 9 years    SEX BY AGE (W… tract    
 5 B01001A_005 Estimate!!Total:!!Male:!!10 to 14 years  SEX BY AGE (W… tract    
 6 B01001A_006 Estimate!!Total:!!Male:!!15 to 17 years  SEX BY AGE (W… tract    
 7 B01001A_007 Estimate!!Total:!!Male:!!18 and 19 years SEX BY AGE (W… tract    
 8 B01001A_008 Estimate!!Total:!!Male:!!20 to 24 years  SEX BY AGE (W… tract    
 9 B01001A_009 Estimate!!Total:!!Male:!!25 to 29 years  SEX BY AGE (W… tract    
10 B01001A_010 Estimate!!Total:!!Male:!!30 to 34 years  SEX BY AGE (W… tract    
# ℹ 2

In [104]:
%%R

library(dplyr)
library(tidyr)

# Reshape the data
reshaped_data <- B03002_vars %>%
  mutate(census_tract = stringr::str_extract(name, "\\d+")) %>%
  pivot_longer(cols = -c(name, census_tract), names_to = "race", values_to = "estimate")

# Print the reshaped data
print(reshaped_data)

# A tibble: 83,658 × 4
   name        census_tract race      estimate                              
   <chr>       <chr>        <chr>     <chr>                                 
 1 B01001A_001 01001        label     Estimate!!Total:                      
 2 B01001A_001 01001        concept   SEX BY AGE (WHITE ALONE)              
 3 B01001A_001 01001        geography tract                                 
 4 B01001A_002 01001        label     Estimate!!Total:!!Male:               
 5 B01001A_002 01001        concept   SEX BY AGE (WHITE ALONE)              
 6 B01001A_002 01001        geography tract                                 
 7 B01001A_003 01001        label     Estimate!!Total:!!Male:!!Under 5 years
 8 B01001A_003 01001        concept   SEX BY AGE (WHITE ALONE)              
 9 B01001A_003 01001        geography tract                                 
10 B01001A_004 01001        label     Estimate!!Total:!!Male:!!5 to 9 years 
# ℹ 83,648 more rows
# ℹ Use `print(n = ...)` to see 

In [105]:
%%R -o nyc_census_data

nyc_census_data <- get_acs(geography = "tract", 
                           state = 'NY',
                           county = c("New York", "Kings", "Queens", "Bronx", "Richmond"),
                           variables = c(
                             total = "B03002_001",
                             hispanic = "B03002_012",
                             white_nonhispanic = "B03002_003",
                             black_nonhispanic = "B03002_004",
                             asian_nonhispanic = "B03002_006",
                             native_nonhispanic = "B03002_005",
                             someother_nonhispanic = "B03002_007",
                             two_or_more_nonhispanic = "B03002_008"
                             
                           ), 
                           year = 2021,
                           survey = "acs5",
                           geometry = FALSE)

# Display the obtained NYC census data
nyc_census_data

# A tibble: 18,616 × 5
   GEOID       NAME                                   variable    estimate   moe
   <chr>       <chr>                                  <chr>          <dbl> <dbl>
 1 36005000100 Census Tract 1, Bronx County, New York total           6661   702
 2 36005000100 Census Tract 1, Bronx County, New York white_nonh…     1066   307
 3 36005000100 Census Tract 1, Bronx County, New York black_nonh…     3157   470
 4 36005000100 Census Tract 1, Bronx County, New York native_non…       39    33
 5 36005000100 Census Tract 1, Bronx County, New York asian_nonh…      177    76
 6 36005000100 Census Tract 1, Bronx County, New York someother_…        0    18
 7 36005000100 Census Tract 1, Bronx County, New York two_or_mor…       18    21
 8 36005000100 Census Tract 1, Bronx County, New York hispanic        2166   258
 9 36005000200 Census Tract 2, Bronx County, New York total           4453   563
10 36005000200 Census Tract 2, Bronx County, New York white_nonh…       93    98
# ℹ 1

Getting data from the 2017-2021 5-year ACS
Using FIPS code '36' for state 'NY'
Using FIPS code '061' for 'New York County'
Using FIPS code '047' for 'Kings County'
Using FIPS code '081' for 'Queens County'
Using FIPS code '005' for 'Bronx County'
Using FIPS code '085' for 'Richmond County'


In [106]:
df = nyc_census_data

In [107]:
df.drop(columns=['moe'], inplace=True)
df

,GEOID,NAME,variable,estimate
1,36005000100,"Census Tract 1, Bronx County, New York",total,6661.0
2,36005000100,"Census Tract 1, Bronx County, New York",white_nonhispanic,1066.0
3,36005000100,"Census Tract 1, Bronx County, New York",black_nonhispanic,3157.0
4,36005000100,"Census Tract 1, Bronx County, New York",native_nonhispanic,39.0
5,36005000100,"Census Tract 1, Bronx County, New York",asian_nonhispanic,177.0
...,...,...,...,...
18612,36085990100,"Census Tract 9901, Richmond County, New York",native_nonhispanic,0.0
18613,36085990100,"Census Tract 9901, Richmond County, New York",asian_nonhispanic,0.0
18614,36085990100,"Census Tract 9901, Richmond County, New York",someother_nonhispanic,0.0
18615,36085990100,"Census Tract 9901, Richmond County, New York",two_or_more_nonhispanic,0.0


In [108]:
%%R -i df -o df2

df2 <- df %>% 
pivot_wider(
    names_from=variable, 
    values_from = c(estimate),
    names_glue = "{variable}_{.value}"
  )

df2

# A tibble: 2,327 × 10
   GEOID      NAME  total_estimate white_nonhispanic_es…¹ black_nonhispanic_es…²
   <chr>      <chr>          <dbl>                  <dbl>                  <dbl>
 1 360050001… Cens…           6661                   1066                   3157
 2 360050002… Cens…           4453                     93                    934
 3 360050004… Cens…           6000                    372                   1088
 4 360050016… Cens…           6038                    243                   2063
 5 360050019… Cens…           2168                    328                    631
 6 360050019… Cens…           1399                    246                    387
 7 360050019… Cens…              0                      0                      0
 8 360050019… Cens…              0                      0                      0
 9 360050020… Cens…           4694                      0                   2055
10 360050020… Cens…           4274                      0                    780
# ℹ 2

In [109]:
df_merge2.head()
#36061003001

,Unnamed: 0,Borough,Borough/Citywide Office (B/CO),District,School,School Name_x,School Category,Program,Language,Language (Translated),...,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY,mergeid
0,0,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,-73.986229,3,10664,0,-73.986229,G5040,L,U,061,36061003001
1,1,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,-73.986229,3,10664,0,-73.986229,G5040,L,U,061,36061003001
2,2,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,-73.986229,3,10664,0,-73.986229,G5040,L,U,061,36061003001
3,3,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,-73.986229,3,10664,0,-73.986229,G5040,L,U,061,36061003001
4,4,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,-73.986229,3,10664,0,-73.986229,G5040,L,U,061,36061003001


In [110]:
df2[df2['GEOID'] == '36061003001']

,GEOID,NAME,total_estimate,white_nonhispanic_estimate,black_nonhispanic_estimate,native_nonhispanic_estimate,asian_nonhispanic_estimate,someother_nonhispanic_estimate,two_or_more_nonhispanic_estimate,hispanic_estimate
1197,36061003001,"Census Tract 30.01, New York County, New York",3718.0,2108.0,236.0,0.0,588.0,0.0,0.0,773.0


In [111]:
final_df = df_merge2.merge(df2, left_on='mergeid', right_on='GEOID')

## 👉 Merge it with your data

hint...`tidycensus` provides you data in long format you may need to pivot the census data from long to wide format before merging it with your data

In [112]:
final_df

,Unnamed: 0,Borough,Borough/Citywide Office (B/CO),District,School,School Name_x,School Category,Program,Language,Language (Translated),...,GEOID_y,NAME_y,total_estimate,white_nonhispanic_estimate,black_nonhispanic_estimate,native_nonhispanic_estimate,asian_nonhispanic_estimate,someother_nonhispanic_estimate,two_or_more_nonhispanic_estimate,hispanic_estimate
0,0,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,36061003001,"Census Tract 30.01, New York County, New York",3718.0,2108.0,236.0,0.0,588.0,0.0,0.0,773.0
1,1,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,36061003001,"Census Tract 30.01, New York County, New York",3718.0,2108.0,236.0,0.0,588.0,0.0,0.0,773.0
2,2,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,36061003001,"Census Tract 30.01, New York County, New York",3718.0,2108.0,236.0,0.0,588.0,0.0,0.0,773.0
3,3,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,36061003001,"Census Tract 30.01, New York County, New York",3718.0,2108.0,236.0,0.0,588.0,0.0,0.0,773.0
4,4,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,36061003001,"Census Tract 30.01, New York County, New York",3718.0,2108.0,236.0,0.0,588.0,0.0,0.0,773.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2746,2746,Bronx,District 75,75,75X811,P.S. X811,Secondary School,Transitional Bilingual Education,Spanish,Español,...,36005012300,"Census Tract 123, Bronx County, New York",4582.0,0.0,2016.0,5.0,25.0,0.0,0.0,2536.0
2747,2747,Bronx,District 75,75,75X811,P.S. X811,Secondary School,Transitional Bilingual Education,Spanish,Español,...,36005012300,"Census Tract 123, Bronx County, New York",4582.0,0.0,2016.0,5.0,25.0,0.0,0.0,2536.0
2748,2748,Bronx,District 75,75,75X811,P.S. X811,Secondary School,Transitional Bilingual Education,Spanish,Español,...,36005012300,"Census Tract 123, Bronx County, New York",4582.0,0.0,2016.0,5.0,25.0,0.0,0.0,2536.0
2749,2749,Bronx,District 75,75,75X811,P.S. X811,Secondary School,Transitional Bilingual Education,Spanish,Español,...,36005012300,"Census Tract 123, Bronx County, New York",4582.0,0.0,2016.0,5.0,25.0,0.0,0.0,2536.0
